# Process : GlobalView - zonal
_process GlobalView xCO2 data_

- **Original**: `/local/data/artemis/observations/globalview/originals`
- **Processed**: `/local/data/artemis/observations/globalview/processed/`
- **Variable**: xCO2
- **Temporal**: Monthly, 198201 - 201712
- **Spatial**: 1x1 degree, interpolated from native grid

This script processes the GLOBALVIEW xCO2 data. 
This is zonally averaged data. 

Last update : 21 January 2022 Val Bennington

In [2]:
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import date

# Future versions of pandas will require you to explicitly register matplotlib converters
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

In [1]:
def process_globalview():
    # Observations directory
    dir_obs = '/local/data/artemis/observations'

    # Surface file
    fl = f'{dir_obs}/globalview/originals/co2_GHGreference.645082325_zonal.txt'

    df = pd.read_csv(fl, sep='\s+', skiprows=73, header=None, index_col=3)

    #df.head()

    # Time vector
    df_time = pd.DataFrame({'year': df[0].values,
                            'month': df[1].values,
                            'day': df[2].values})
    time = pd.to_datetime(df_time)

    # xCO2 
    xco2 = df[4]
    xco2_uncert = df[5]

    #plt.plot(time, xco2)

    ## Create NetCDF File of raw data
    #the NOAA MLB product uses ~weekly data. there are 48 equal time steps each year (~7.6 days)

    ds_out = xr.Dataset(
        {
        'xco2':(['time'], xco2, 
                {'long_name':'mixing ratio of atmospheric CO2',
                'units':'ppmv'}),
        'xco2_uncertainty':(['time'], xco2_uncert, 
                {'long_name':'mixing ratio of atmospheric CO2 uncertainty',
                'units':'ppmv'}),
         'year' : (['time'], df[0].values,
                  {'long_name':'years'}),
         'month' : (['time'], df[1].values,
                  {'long_name':'month'}),
         'day' : (['time'], df[2].values,
                  {'long_name':'days'})
        },

        coords={
        'time': (['time'], time)
        },
        attrs={'contact':'Luke Gloege (gloege@ldeo.columbia.edu)',
               'creation_date':f'{date.today()}',
               'notes':'raw data was not manipulated',
               'original_data':'https://www.esrl.noaa.gov/gmd/ccgg/mbl/'})

    #ds_out.info()

    ## Save native data in netcdf

    #ds_out.to_netcdf(f'{dir_obs}/globalview/processed/xco2_zonal_weekly_globalview_197901-201812.nc')

    #ds_out['time']

    ## Sample the data monthly

    # Define Dates
    dates = pd.date_range(start='1979-01-01T00:00:00.000000000', 
                          end='2018-12-01T00:00:00.000000000',freq='MS')+ np.timedelta64(14, 'D')
    #print(f'dates shape = {np.shape(dates)}')

    ds_out_monthly = xr.Dataset(
        {
        'xco2':(['time'], ds_out['xco2'].resample(time='1MS').mean().values, 
                {'long_name':'mixing ratio of atmospheric CO2',
                'units':'ppmv',
                'notes':'monthly averages from weekly data'})
        },

        coords={
        'time': (['time'], dates)
        },
        attrs={'contact':'Luke Gloege (gloege@ldeo.columbia.edu)',
               'creation_date':f'{date.today()}',
               'notes':'these are monthly averages from raw weekly data',
               'original_data':'https://www.esrl.noaa.gov/gmd/ccgg/mbl/'})

    #ds_out_monthly.info()

    #ds_out_monthly.to_netcdf(f'{dir_obs}/globalview/processed/xco2_zonal_mon_globalview_197901-201812.nc')

    ## Repeat monthly data to be 1x1

    lon = np.arange(0.5,360,1)
    lat = np.arange(-89.5,90,1)
    ds_bc = xr.DataArray(np.zeros([len(lon),len(lat)]), coords=[('lon', lon),('lat', lat)])
    ds_data, ds_mask = xr.broadcast(ds_out_monthly, ds_bc)

    ds_data = ds_data.transpose('time', 'lat', 'lon')

    return ds_data
    #file_name = f'{dir_obs}/globalview/processed/xco2_1x1_mon_globalview_197901-201812.nc'
    #ds_data.to_netcdf(file_name)               

In [40]:
ds_data = process_globalview()

In [ ]:
#file_name = f'{dir_obs}/globalview/processed/xco2_1x1_mon_globalview_197901-201812.nc'
#ds_data.to_netcdf(file_name)  

# Other snippets

In [ ]:
# Surface file
#fl = f'{dir_obs}/globalview/originals/co2_GHGreference.424818983_surface.txt'
#df = pd.read_csv(fl, sep='\s+', skiprows=78, header=None, index_col=0)
#df.index.name = 'decimal-year'
#df.head()
#df = df.iloc[:,::2]
#df.head()
#df.columns = 'CO2'
#df.index.name = 'decimal-year'

#df.index.name = 'decimal-year'
#df.head()
#df = df.iloc[:,::2]
#df.head()
#df.columns = 'CO2'
#df.index.name = 'decimal-year'

In [ ]:
#ds_hind = xr.open_dataset(f'{dir_hindcast}/CESM-hindcast_XCO2_1x1_198201-201512.nc')
#plt.plot(ds_data['time'], ds_data['XCO2'][:,0,0])
#plt.plot(ds_hind['time'], ds_hind['XCO2'][:,50,50])